### plan
1. use pretrained `resnet50` as feature extractor
2. pre calculate features of training and validation data
3. fine tune fc layers

> start using `tensorflow` as backend

In [1]:
%load_ext autoreload
%autoreload 2

from theano.sandbox import cuda
cuda.use('gpu0')

from keras.applications.vgg16 import VGG16
from keras.preprocessing.image import ImageDataGenerator
from keras.utils.np_utils import to_categorical

from keras.models import Model, Sequential
from keras.layers import Input, Dense, Dropout, BatchNormalization, Flatten, Convolution2D, MaxPooling2D
from keras.optimizers import Adam

import h5py
import numpy as np

Using gpu device 0: GeForce GTX TITAN X (CNMeM is disabled, cuDNN 5103)
/home/lukaka/anaconda2/envs/py3/lib/python3.5/site-packages/theano/sandbox/cuda/__init__.py:600: UserWarning: Your cuDNN version is more recent than the one Theano officially supports. If you see any problems, try updating Theano or downgrading cuDNN to version 5.
  warnings.warn(warn)
Using Theano backend.


In [2]:
TRAIN_PATH = './data/redux/train'
VALID_PATH = './data/redux/validation'

### load base model

In [3]:
base_model = VGG16(input_shape=(3, 224,224))

In [4]:
base_model.layers

In [5]:
maxpool_idx = [id for id, layer in enumerate(base_model.layers) if isinstance(layer, MaxPooling2D)]
maxpool_idx

[3, 6, 10, 14, 18]

In [6]:
conv_layers = base_model.layers[:maxpool_idx[-1] + 1]

In [7]:
conv_model = Sequential(conv_layers)

### pre compute features and pre process labels

In [8]:
def get_generator(path):
    gen = ImageDataGenerator()
    return gen.flow_from_directory(path, target_size=(224, 224), class_mode='categorical', shuffle=True)

In [9]:
trn_gen = get_generator(TRAIN_PATH)
val_gen = get_generator(VALID_PATH)

Found 22500 images belonging to 2 classes.
Found 2500 images belonging to 2 classes.


In [10]:
trn_features = conv_model.predict_generator(trn_gen, trn_gen.nb_sample)
val_features = conv_model.predict_generator(val_gen, val_gen.nb_sample)

In [13]:
trn_labels = to_categorical(trn_gen.classes)
val_labels = to_categorical(val_gen.classes)

### save features and labels ndarray as h5 files

In [14]:
def save_array(file_name, ndarray):
    with h5py.File(file_name, 'w') as f:
        f.create_dataset('data', data=ndarray)

In [15]:
save_array('trn_features.h5', trn_features)
save_array('val_features.h5', val_features)
save_array('trn_labels.h5', trn_labels)
save_array('val_labels.h5', val_labels)

### load features from h5 files

In [95]:
trn_features = load_array('trn_features.h5')
val_features = load_array('val_features.h5')
r
trn_labels = load_array('trn_labels.h5')
val_labels = load_array('val_labels.h5')

### build fc layers
* feature input `(1,1,2048)`
* flatten
* fc1
* fc2
* output (2)

In [37]:
for idx, layer in enumerate(base_model.layers):
    if isinstance(layer, Dense):
        dense_idx = idx
        break
dense_idx

20

In [38]:
dense_layers = base_model.layers[dense_idx: -1]
dense_layers

In [39]:
fc1, fc2 = dense_layers

In [40]:
feature_output_shape = conv_layers[-1].output_shape[1:]
feature_output_shape

(7, 7, 512)

In [41]:
p = 0.

In [42]:
feature_input = Input(shape=feature_output_shape)
x = Flatten()(feature_input)

x = fc1(x)
x = Dropout(p)(x)
# x = BatchNormalization()(x)

x = fc2(x)
x = Dropout(p)(x)
# x = BatchNormalization()(x)

output = Dense(2, activation='softmax')(x)
model = Model(feature_input, output)

opt = Adam(lr=1e-4)
model.compile(optimizer=opt, loss='categorical_crossentropy', metrics=['accuracy'])

In [43]:
for layer in model.layers[:-1]:
    layer.trainable = False

In [44]:
model.summary()

____________________________________________________________________________________________________
Layer (type)                     Output Shape          Param #     Connected to                     
input_5 (InputLayer)             (None, 7, 7, 512)     0                                            
____________________________________________________________________________________________________
flatten_3 (Flatten)              (None, 25088)         0           input_5[0][0]                    
____________________________________________________________________________________________________
fc1 (Dense)                      (None, 4096)          102764544   flatten_3[0][0]                  
____________________________________________________________________________________________________
dropout_5 (Dropout)              (None, 4096)          0           fc1[1][0]                        
___________________________________________________________________________________________

In [45]:
model.fit(trn_features, trn_labels, nb_epoch=10, batch_size=64, 
          validation_data=(val_features, val_labels), 
          shuffle=False,
          verbose=2)

Train on 22500 samples, validate on 2500 samples
Epoch 1/10
33s - loss: 8.0602 - acc: 0.4988 - val_loss: 8.0784 - val_acc: 0.4988
Epoch 2/10
32s - loss: 8.0569 - acc: 0.5001 - val_loss: 8.0784 - val_acc: 0.4988
Epoch 3/10
32s - loss: 8.0569 - acc: 0.5001 - val_loss: 8.0784 - val_acc: 0.4988
Epoch 4/10
32s - loss: 8.0569 - acc: 0.5001 - val_loss: 8.0784 - val_acc: 0.4988
Epoch 5/10


KeyboardInterrupt: 